In [3]:
import pandas as pd

bike_data = pd.read_csv('data/citibike-tripdata.csv')
bike_data.info()

bike_copy = bike_data.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station id         299831 non-null  float64
 3   start station name       299831 non-null  object 
 4   start station latitude   300000 non-null  float64
 5   start station longitude  300000 non-null  float64
 6   end station id           299831 non-null  float64
 7   end station name         299831 non-null  object 
 8   end station latitude     300000 non-null  float64
 9   end station longitude    300000 non-null  float64
 10  bikeid                   300000 non-null  int64  
 11  usertype                 300000 non-null  object 
 12  birth year               300000 non-null  int64  
 13  gender                   300000 non-null  int64  
dtypes: f

In [6]:
bike_copy['start station id'].value_counts()

start station id
281.0     1928
2006.0    1909
519.0     1873
514.0     1845
426.0     1835
          ... 
3432.0      15
3599.0      11
2005.0       9
3704.0       7
3583.0       6
Name: count, Length: 759, dtype: int64

In [7]:
bike_copy['bikeid'].value_counts()

bikeid
33887    107
34358    104
34159    103
34258    103
33916    102
        ... 
18587      1
17324      1
29593      1
33157      1
25794      1
Name: count, Length: 10004, dtype: int64

In [30]:
# Какой тип клиентов (столбец usertype) является преобладающим — Subscriber или Customer? 
# В качестве ответа запишите долю клиентов преобладающего типа среди общего количества клиентов. Ответ округлите до сотых.

bike_copy['usertype'].value_counts()
usertype_Subscriber = bike_copy['usertype'] == 'Subscriber'
usertype_Customer = bike_copy['usertype'] == 'Customer'

user_count = bike_copy[usertype_Subscriber].shape[0] / bike_copy.shape[0]
print(round(user_count, 2))

0.77


In [32]:
bike_copy['gender'].value_counts()

gender
1    183582
2     74506
0     41912
Name: count, dtype: int64

In [34]:
print(bike_copy['start station id'].nunique())
print(bike_copy['end station id'].nunique())

759
765


In [46]:
2018 - bike_copy['birth year'].values.max()

16

In [59]:
print(bike_copy['start station id'].value_counts().tail(1))
bike_copy['start station name'][3583.0]

start station id
3583.0    6
Name: count, dtype: int64


'Leonard St & Church St'

In [72]:
print(bike_copy['end station id'].value_counts().head(1))
bike_copy['end station name'][72.0]

end station id
426.0    1963
Name: count, dtype: int64


'W 25 St & 6 Ave'

In [73]:
bike_copy.drop(['start station id', 'end station id'], axis=1, inplace=True)
print(bike_copy.shape[1])

12


In [76]:
bike_copy['age'] = 2018 - bike_copy['birth year']
# display(bike_copy['age'])
age_mask = bike_copy['age'] > 60
bike_copy[age_mask].shape[0]

11837

In [94]:
# Создайте признак длительности поездки trip duration. 
# Для этого вычислите интервал времени между временем окончания поездки (stoptime) и её началом (starttime). 
# Сколько целых минут длилась поездка под индексом 3 в таблице?

bike_copy['starttime'] = pd.to_datetime(bike_copy['starttime'])
bike_copy['stoptime'] = pd.to_datetime(bike_copy['stoptime'])
bike_copy['trip duration'] = (bike_copy['stoptime'] - bike_copy['starttime'])
bike_copy.loc[3, 'trip duration']


Timedelta('0 days 00:07:16.837000')

In [99]:
# Создайте «признак-мигалку» weekend, который равен 1, если поездка начиналась в выходной день (суббота или воскресенье), 
# и 0 — в противном случае. Выясните, сколько поездок начиналось в выходные.

bike_copy['day of week'] = bike_copy['starttime'].dt.dayofweek
# display(bike_copy['day of week'])
bike_copy['weekend'] = bike_copy['day of week'].apply(lambda x: 1 if x in [5, 6] else 0)
display(bike_copy['weekend'].value_counts())

weekend
0    184865
1    115135
Name: count, dtype: int64

In [105]:
# Создайте признак времени суток поездки time_of_day. Время суток будем определять из часа начала поездки. Условимся, что:
# поездка совершается ночью (night), если её час приходится на интервал от 0 (включительно) до 6 (включительно) часов;
# поездка совершается утром (morning), если её час приходится на интервал от 6 (не включительно) до 12 (включительно) часов;
# поездка совершается днём (day), если её час приходится на интервал от 12 (не включительно) до 18 (включительно) часов;
# поездка совершается вечером (evening), если её час приходится на интервал от 18 (не включительно) до 23 часов (включительно).
# Во сколько раз количество поездок, совершённых днём, больше, 
# чем количество поездок, совёршенных ночью, за представленный в данных период времени? 
# Ответ округлите до целых.

bike_copy['hour'] = bike_copy['starttime'].dt.hour

def time_of_day(hour):
    if 0 <= hour <= 6:
        return 'night'
    elif 6 < hour <= 12:
        return 'morning'
    elif 12 < hour <= 18:
        return 'day'
    elif 18 < hour <= 23:
        return 'evening'
    
bike_copy['time_of_day'] = bike_copy['hour'].apply(time_of_day)
display(bike_copy['time_of_day'].value_counts())

time_of_day
day        143012
morning     95530
evening     46373
night       15085
Name: count, dtype: int64